In [19]:
import requests
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv()

# Get Strava Access Token
access_token = os.getenv("STRAVA_ACCESS_TOKEN")

response = requests.get(
    "https://www.strava.com/api/v3/athlete/activities",
    headers={"Authorization": f"Bearer {access_token}"},
    params={
        "per_page": 100,  # Fetch 100 activities at once
        "page": 1         # Page 1
    }
)

#print(response.json())

# Assuming `activities` is your list of activity JSON from Strava
def display_summary(activities):
    for activity in activities:
        name = activity.get("name")
        sport_type = activity.get("sport_type")
        distance_km = round(activity.get("distance", 0) / 1000, 2)
        time_min = round(activity.get("moving_time", 0) / 60, 1)
        date = datetime.strptime(activity["start_date"], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d")
        heart = f"{activity['average_heartrate']} bpm" if activity.get("average_heartrate") else "n/a"

        print(f"🏋️ {name} | {sport_type} | {distance_km} km | {time_min} min | {date} | HR: {heart}")

# Example usage
display_summary(response.json())  # if you have raw response

🏋️ Afternoon workout 💪 | WeightTraining | 0.0 km | 48.5 min | 2025-04-24 | HR: n/a
🏋️ Afternoon workout 💪 | WeightTraining | 0.0 km | 111.5 min | 2025-04-23 | HR: n/a
🏋️ Evening Run | Run | 5.23 km | 30.1 min | 2025-04-22 | HR: 185.4 bpm
🏋️ Evening workout 🏋️ | WeightTraining | 0.0 km | 94.6 min | 2025-04-21 | HR: n/a
🏋️ Morning Run | Run | 5.1 km | 29.3 min | 2025-04-21 | HR: 183.4 bpm
🏋️ Morning workout ☀️ | WeightTraining | 0.0 km | 68.6 min | 2025-04-19 | HR: n/a
🏋️ Evening workout 🏋️ | WeightTraining | 0.0 km | 79.0 min | 2025-04-18 | HR: n/a
🏋️ Lunch Run | Run | 4.04 km | 22.2 min | 2025-04-17 | HR: 190.4 bpm
🏋️ Afternoon workout 💪 | WeightTraining | 0.0 km | 108.2 min | 2025-04-14 | HR: n/a
🏋️ Lunch Run | Run | 5.1 km | 29.2 min | 2025-04-13 | HR: 190.7 bpm
🏋️ Morning workout ☀️ | WeightTraining | 0.0 km | 91.2 min | 2025-04-12 | HR: n/a
🏋️ Afternoon workout 💪 | WeightTraining | 0.0 km | 101.2 min | 2025-04-11 | HR: n/a
🏋️ Paris Run🥖🇫🇷 | Run | 5.06 km | 29.4 min | 2025-04-05 | H

In [20]:
import json

with open("strava_activities.json", "w") as f:
    json.dump(response.json(), f, indent=4)

print("Exported to strava_activities.json")

Exported to strava_activities.json


In [21]:
import csv
from datetime import datetime

activities = response.json()

with open("strava_activities_detailed.csv", "w", newline="") as csvfile:
    fieldnames = [
        "name", "sport_type", "distance_km", "moving_time_min", "elapsed_time_min",
        "elevation_gain_m", "average_heartrate", "max_heartrate", "average_speed_kmh",
        "start_date", "start_latlng", "end_latlng", "kudos_count", "suffer_score"
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for activity in activities:
        writer.writerow({
            "name": activity.get("name"),
            "sport_type": activity.get("sport_type"),
            "distance_km": round(activity.get("distance", 0) / 1000, 2),
            "moving_time_min": round(activity.get("moving_time", 0) / 60, 1),
            "elapsed_time_min": round(activity.get("elapsed_time", 0) / 60, 1),
            "elevation_gain_m": activity.get("total_elevation_gain", 0),
            "average_heartrate": activity.get("average_heartrate", "n/a"),
            "max_heartrate": activity.get("max_heartrate", "n/a"),
            "average_speed_kmh": round(activity.get("average_speed", 0) * 3.6, 2),
            "start_date": datetime.strptime(activity["start_date"], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d"),
            "start_latlng": activity.get("start_latlng", []),
            "end_latlng": activity.get("end_latlng", []),
            "kudos_count": activity.get("kudos_count", 0),
            "suffer_score": activity.get("suffer_score", "n/a"),
        })

print("Exported to strava_activities_detailed.csv")

Exported to strava_activities_detailed.csv


In [34]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# Set up OpenAI client
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

# Load your Strava activities
with open("strava_activities.json") as f:
    activities = json.load(f)

# Summarize last 10 workouts
summary = "\n".join([
    f"{a['name']} - {a['sport_type']} - {round(a['distance'] / 1000, 2)} km on {a['start_date'][:10]}"
    for a in activities[:10]
])

# Build the OpenAI prompt
prompt_assistant = f"""
You are a Fitness & Nutrition AI Assistant.

Here is the user's recent training and nutrition data over the last few days:

{summary}

Goals:
- Analyze the balance between calories burned and calories eaten.
- Evaluate macro balance (protein, carbs, fats) per day.
- Suggest daily adjustments (e.g., more protein, less carbs).
- Create a full day meal plan optimized for fitness goals.
- Recipes should be simple and healthy.

Consider the user is 180 cm tall and weighs 80 kg. They are actively training.

Important: Please clearly list the meals with calories for each meal (e.g., "Breakfast: 400 kcal").
"""

# Create chat completion
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": prompt_assistant}
    ]
)

# Get the assistant response text
assistant_output = response.choices[0].message.content

# Save the assistant output to a JSON file
output_data = {
    "assistant_response": assistant_output
}

with open("meal_plan_from_openai.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

print("✅ Saved assistant meal plan to 'meal_plan_from_openai.json'")

✅ Saved assistant meal plan to 'meal_plan_from_openai.json'


# Personal Trainer Based on Strava API Data

In [33]:
with open("strava_activities.json") as f:
    activities = json.load(f)

# Summarize all activities (dynamic!)
summary = "\n".join([
    f"{a['name']} - {a['sport_type']} - {round(a.get('distance', 0) / 1000, 2)} km on {a['start_date'][:10]}"
    for a in activities
])

# Build the training coach prompt
prompt_coach = f"""
You are an experienced personal coach and professional training planner AI.

Here is the user's complete workout history from Strava:

{summary}

User details:
- Height: 180 cm
- Weight: 80 kg
- Training Goal: Improve endurance and strength.
- If there's a gym session, then no run and vice versa

Your task:
- Analyze the complete workout history
- Identify training gaps, overtraining risks, or missing elements
- Create a detailed 7-day training plan
- The plan should include specific workout types, target duration, intensity levels, and recovery days
- Recommend any modifications for injury prevention
- Ensure the plan supports gradual performance progression
- Be realistic and motivating

Start your response with an encouraging message!
"""

# Now create chat completion using the client
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": prompt_coach}
    ]
)

# Output the generated training plan
print(response.choices[0].message.content)

Hey there, fantastic to see your dedication and consistency with your workouts! You're doing an amazing job balancing strength training and endurance, which is a great foundation for overall fitness. Let’s take a closer look at your training history and craft a personalized 7-day plan to help you continue progressing towards your endurance and strength goals safely and effectively.

### Analysis:

**Strengths:**
- Strong dedication to both weight training and running.
- Consistent exercise routine, with regular running and weightlifting sessions.
- Mixed variety of workouts including runs of varied distances and interval training.

**Potential Gaps/Improvements:**
- Limited recovery or rest days, which may increase the risk of overtraining injuries.
- Runs are mostly steady-state; little mention of speed work or varied terrain, apart from interval sessions.
- No mention of mobility or flexibility work.

### Recommendations:

1. **Rest and Recovery:** Ensure you have at least one dedica